In [5]:
"""from datasets import load_dataset
import pandas as pd

scicite = load_dataset('hrithikpiyush/acl-arc')

# Convert to a pandas DataFrame
df = pd.DataFrame({
    'text': scicite['train']['text'],
    'label': scicite['train']['section_name'],
})

# Save to CSV
df.to_csv('aclarc.csv', index=False)"""

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/makarwuckert/.cache/huggingface/datasets/hrithikpiyush___json/hrithikpiyush--acl-arc-0e1cc5b7c51f083f/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
import pandas as pd

df = pd.read_csv('../datasets/aclarc.csv')
df.head(3)

,text,label
0,"Thus , over the past few years , along with ad...",introduction
1,"This was done by MERT optimization ( Och , 200...",experiments
2,"She evaluates 3,000 German verbs with a token ...",NaN


In [7]:
df.dropna(inplace=True)
df.head(3)

,text,label
0,"Thus , over the past few years , along with ad...",introduction
1,"This was done by MERT optimization ( Och , 200...",experiments
5,This is roughly an 11 % relative reduction in ...,conclusion


In [10]:
df.shape

(1223, 2)

In [11]:
"""
Now we do Data Preprocessing.
"""
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

regexp = RegexpTokenizer("[\w']+")

def convert_to_lowercase(text):
    return text.lower()
def remove_whitespace(text):
    return text.strip()
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "") 
    return text.translate(str.maketrans("", "", punct_str))
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)
def remove_http(text):
    http = "https?://\S+|www\.\S+" 
    pattern = r"({})".format(http) 
    return re.sub(pattern, "", text)
# Stopwords
stops = stopwords.words("english") 
addstops = ["among", "onto", "shall", "thrice", "thus", "twice", "unto", "us", "would"] 
allstops = stops + addstops
def remove_stopwords(text):
    return " ".join([word for word in regexp.tokenize(text) if word not in allstops])
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha

In [12]:
def text_normalizer(text):
    text = convert_to_lowercase(text)
    text = remove_whitespace(text)
    text = re.sub('\n' , '', text) 
    text = re.sub('\[.*?\]', '', text) 
    text = remove_http(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = remove_stopwords(text)
    text = discard_non_alpha(text)
    return text

In [13]:
df['text'] = df['text'].apply(text_normalizer)

In [14]:
df['text'][0]

'past years along advances use learning statistical methods acquisition full parsers collins charniak charniak ratnaparkhi significant progress made use statistical learning methods recognize shallow parsing patterns syntactic phrases words participate syntactic relationship church ramshaw marcus argamon et al cardie pierce munoz et al punyakanok roth buchholz et al tjong kim sang buchholz'

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,2), max_features=15000, smooth_idf=True, sublinear_tf=True)
df_feature = vectorizer.fit_transform(df['text'])
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_feature, df['label'], test_size=0.2, random_state=42)

In [16]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

#bei max_depth = 9, schon 99% accuracy
dt_model = DecisionTreeClassifier()
dt_model.fit(df_features_train, df_target_train)
target_prediction = dt_model.predict(df_features_test)

In [17]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(df_target_test, target_prediction))

              precision    recall  f1-score   support

  conclusion       0.56      0.26      0.36        19
 experiments       0.38      0.62      0.47        42
introduction       0.67      0.77      0.71        91
      method       0.50      0.37      0.42        19
related work       0.69      0.45      0.54        74

    accuracy                           0.58       245
   macro avg       0.56      0.49      0.50       245
weighted avg       0.60      0.58      0.57       245



In [18]:
accuracy_score = accuracy_score(df_target_test, target_prediction)

In [19]:
accuracy_score

0.5755102040816327